# PORTAFOLIOS TIPO MARKOWIT

En este notebook se generarán portofolios con acciones pertenecientes al S&P 500 con el fin de evaluar su desempeño por fuera de los datos de optimización.  Se utilizarán ventanas de optimización de 2 años y de pruebas de 6 meses

### Carga de Librerias

In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import scipy
from scipy.optimize import minimize
import scipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Declaración de Funciones

In [33]:
def annualize_rets(r, periods_per_year):
    "anualiza retorns desde subperiodos de año"
    compounded_growth = np.prod(1+r)
    n_periods = r.shape[0]
    return compounded_growth**(periods_per_year / n_periods)-1

In [47]:
def carga_datos(path):
    df = pd.read_parquet(path)
    df.set_index('Date', inplace=True)
    return df

### Carga de Datos

In [48]:
datos1 = carga_datos('./Data/train/opt-20130101-20141231.gzip')

In [49]:
datos1.shape

(504, 465)

In [50]:
datos1.head()

Stock,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,0.022960,0.035556,0.001935,0.031681,0.007179,0.018601,0.019827,0.038496,0.017516,0.046125,...,0.025084,0.034300,0.024957,0.022974,0.054252,0.014760,0.011446,0.015602,0.041974,0.027570
2013-01-03,0.003582,-0.022890,0.000000,-0.012622,-0.002070,-0.017531,0.038065,-0.003620,-0.015389,-0.016137,...,-0.002922,-0.001887,-0.001803,-0.002715,0.000000,-0.003273,0.006254,0.014328,0.000977,0.001364
2013-01-04,0.019748,0.078331,0.015589,-0.027855,0.006682,-0.002974,-0.006011,0.005522,0.010066,-0.017787,...,0.003663,-0.014047,0.004630,0.008661,-0.011126,-0.001459,0.010950,0.005097,-0.008049,0.028611
2013-01-07,-0.007233,0.007468,-0.003396,-0.005882,0.003205,-0.011931,0.008164,-0.004336,-0.004983,0.003057,...,-0.010584,-0.004110,-0.011578,0.003435,0.016877,-0.013153,-0.006294,0.002463,0.005655,-0.004857
2013-01-08,-0.007991,0.021563,-0.016356,0.002691,-0.001369,-0.003019,0.000300,0.005806,0.005272,-0.010317,...,0.001844,-0.020358,0.006255,-0.011492,-0.001383,-0.012218,-0.041980,0.001445,0.000733,-0.019964


In [51]:
cov1 = datos1.cov()

In [52]:
cov1

Stock,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION
Stock,,,,,,,,,,,,,,,,,,,,,
A,0.000194,0.000095,0.000064,0.000033,0.000045,0.000058,0.000062,0.000067,0.000072,0.000082,...,0.000025,0.000081,0.000048,0.000065,0.000087,0.000090,0.000069,0.000068,0.000080,0.000084
AAL,0.000095,0.000555,0.000073,0.000037,0.000061,0.000104,0.000049,0.000069,0.000106,0.000081,...,0.000027,0.000069,0.000030,0.000071,0.000080,0.000099,0.000083,0.000066,0.000100,0.000085
AAP,0.000064,0.000073,0.000241,0.000026,0.000038,0.000063,0.000059,0.000044,0.000067,0.000050,...,0.000030,0.000055,0.000030,0.000042,0.000063,0.000053,0.000052,0.000045,0.000048,0.000055
AAPL,0.000033,0.000037,0.000026,0.000255,0.000010,0.000029,0.000018,0.000025,0.000029,0.000048,...,0.000010,0.000020,0.000017,0.000024,0.000041,0.000036,0.000031,0.000052,0.000038,0.000035
ABC,0.000045,0.000061,0.000038,0.000010,0.000095,0.000065,0.000044,0.000037,0.000048,0.000040,...,0.000024,0.000033,0.000029,0.000042,0.000044,0.000042,0.000036,0.000040,0.000033,0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,0.000090,0.000099,0.000053,0.000036,0.000042,0.000078,0.000046,0.000059,0.000092,0.000082,...,0.000022,0.000087,0.000058,0.000063,0.000089,0.000251,0.000081,0.000060,0.000064,0.000081
YUM,0.000069,0.000083,0.000052,0.000031,0.000036,0.000061,0.000041,0.000045,0.000068,0.000056,...,0.000028,0.000051,0.000044,0.000049,0.000057,0.000081,0.000184,0.000049,0.000043,0.000064
ZBH,0.000068,0.000066,0.000045,0.000052,0.000040,0.000068,0.000061,0.000048,0.000054,0.000056,...,0.000031,0.000032,0.000041,0.000056,0.000076,0.000060,0.000049,0.000144,0.000060,0.000055


In [53]:
anual_returns = annualize_rets(datos1, 244)

In [54]:
anual_returns

Stock
A       0.185978
AAL     0.954732
AAP     0.468512
AAPL    0.224599
ABC     0.448193
          ...   
XYL     0.196588
YUM     0.066594
ZBH     0.306462
ZBRA    0.388276
ZION    0.154654
Length: 465, dtype: float64